----
# Model Evaluation
-------

### Summary:
Evaluating the performance of base logistic regression, logistic regression with feature extraction and CNN with real world images.

### Data Overview:
- **Dataset:** Real World images captured using Teachable Machine
- **Number of Samples:** Approximately 300 images per letter, only to use two images per letter to test the models.

### Notebook Overview:
- **Data Loading:**
    - Load the Real Dataset using keras
    - Randomly select 2 images per class

- **Model Testing**:
   - Pass images into model.predict 
   - Store results in a dataframe

- **Model Evaluation**:
   - Assess the accuracy of the models predictions

## Set Up
-----

In [39]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from skimage.feature import hog, local_binary_pattern
import numpy as np
import pandas as pd
import joblib
import re 
import random


## Utility Functions
-----


In [40]:
alphabet = ['a','b','c','d','e','f','g','h','i','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y']

In [41]:
# Randomly picks an image for a given letter
def get_test_imgs(letter):
    '''
    Overview:
    Gets a random image for a given letter.

    Arguments:
        - letter -> Letter (NOT J/Z as both of these letters do not exist in dataset)

    Output:
        - Random image for a given letteer

    '''
    if letter.lower() in ('j', 'z'):
        raise ValueError("The input letter cannot be 'j' or 'z'.")
    else:
        rand_idx = random.randint(1, 300)
        image_path = f'../../data/my_imgs/real_world_imgs/{letter.upper()}/{rand_idx}.jpg'
        return image_path

In [42]:
def extract_features(image):
  '''
    Overview:
    Extracts HOG, LBP, and color histogram features from a 28x28 image.

    Arguments:
        - single image

    Output:
        - feature vector for a given image
  '''
  # Reshape the image if it's flattened
  image = image.reshape(28, 28)

  # Histogram of Orientated Gradients (HOG) Features
  hog_features = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2))

  # LBP Features (Uniform LBP with 8 neighbors)
  lbp_features = local_binary_pattern(image, P=8, R=1, method='uniform')
  lbp_hist, bin_edges = np.histogram(lbp_features, bins=np.arange(0, lbp_features.max() + 2), density=True)

  # Color Histogram Features (8 bins per channel) -> distribution of colour (greyscale in this case)
  color_hist, bin_edges = np.histogram(image, bins=8, range=(0, 255), density=True)

  # Combining features into a single vector
  feature_vectors = np.concatenate((hog_features, lbp_hist, color_hist))

  return feature_vectors


In [43]:
def log_reg_process_image(letter, model):
    '''
        Overview:
        Processes image and passess it to .predict for the given model

        Arguments:
            - letter -> Letter (NOT J/Z as both of these letters do not exist in dataset)
            - model -> to do prediction
        Output:
            - actual letter and predicted letter
    '''  
    if letter.lower() in ('j', 'z'):
        raise ValueError("The input letter cannot be 'j' or 'z'.")
    else:

        image_path = get_test_imgs(letter)

        input_image = load_img(image_path, target_size=(28, 28))
        input_image = input_image.convert('L')
        image_array = img_to_array(input_image).astype(float)
        input_img = image_array.reshape(1,28*28)
        prediction = alphabet[model.predict(input_img)[0]]

    return letter, prediction

In [44]:
def log_reg_fe_process_image(letter, model):
    '''
        Overview:
        Processes image and passess it to .predict for the given model

        Arguments:
            - letter -> Letter (NOT J/Z as both of these letters do not exist in dataset)
            - model -> to do prediction
        Output:
            - actual letter and predicted letter
    '''  
    if letter.lower() in ('j', 'z'):
        raise ValueError("The input letter cannot be 'j' or 'z'.")
    else:
        image_path = get_test_imgs(letter)
        input_image = load_img(image_path, target_size=(28, 28))
        input_image = input_image.convert('L')
        image_array = img_to_array(input_image).astype(float)
        input_img = image_array.reshape(1,28*28)

        feature_vec = extract_features(input_img)
        feature_vec =feature_vec.reshape(1,-1)

        prediction = alphabet[model.predict(feature_vec)[0]]

        return letter, prediction


In [45]:
def cnn_process_image(letter, model):
    '''
        Overview:
        Processes image and passess it to .predict for the given model

        Arguments:
            - letter -> Letter (NOT J/Z as both of these letters do not exist in dataset)
            - model -> to do prediction
        Output:
            - actual letter and predicted letter
    '''  
    if letter.lower() in ('j', 'z'):
        raise ValueError("The input letter cannot be 'j' or 'z'.")
    else:
        image_path = get_test_imgs(letter)
        input_image = load_img(image_path, target_size=(28, 28))
        input_image = input_image.convert('L')
        image_array = img_to_array(input_image).astype(float)
        input_img = image_array.reshape(1, image_array.shape[0], image_array.shape[1], image_array.shape[2])
        soft_pred = model.predict(input_img)
        pred_y = np.argmax(soft_pred, axis=1)
        
        prediction = alphabet[pred_y[0]]

        return letter, prediction

In [46]:
def model_pred (model_name, model, df):
    '''
        Overview:
        To return a dataframe with all results for a given model

        Arguments:
            - model_name -> to store in df
            - model -> to make prediction
            - df -> to store results
        Output:
            - dataframe of results
    '''  

    alphabet = ['a','b','c','d','e','f','g','h','i','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y']

    for letter in alphabet:
        for i in range(2):  # Get two rand images per letter
            # looking for name to decide which processor to use
            if re.search('.*fea.*eng.*', model_name.lower()):
                letter, prediction = log_reg_fe_process_image(letter, model)
            elif re.search('.*log.*reg.*', model_name.lower()):
                letter, prediction = log_reg_process_image(letter, model)
            elif re.search('.*cnn.*', model_name.lower()):
                letter, prediction = cnn_process_image(letter, model)
            else:
                print(f'{model_name}: Model not expected')
            correct = np.where(letter == prediction, 'Y','N')
            row = { 'Model': model_name, 'Actual Letter': letter, 'Predicted Letter': prediction, 'Correct' : correct}
            df.loc[len(df)] = row

    # Return the DataFrame to see the resulrts
    return(df)

## Initialise dataframes to store results for each model
---

In [47]:
# Base Logistic Regression
base_lr_df = pd.DataFrame(columns=['Model', 'Actual Letter',  'Predicted Letter', 'Correct'])
# Logistic Regression with Feature engineering
fe_lr_df = pd.DataFrame(columns=['Model', 'Actual Letter',  'Predicted Letter', 'Correct'])
# CNN
cnn_df = pd.DataFrame(columns=['Model', 'Actual Letter',  'Predicted Letter', 'Correct'])

## Logistic Regression Testing
----

In [48]:
log_reg_model = joblib.load('../../model/my_models/log_reg_basic_model.pkl')

In [49]:
model_pred('Base Log Reg', log_reg_model, base_lr_df)

,Model,Actual Letter,Predicted Letter,Correct
0,Base Log Reg,a,t,N
1,Base Log Reg,a,v,N
2,Base Log Reg,b,s,N
3,Base Log Reg,b,i,N
4,Base Log Reg,c,h,N
5,Base Log Reg,c,f,N
6,Base Log Reg,d,t,N
7,Base Log Reg,d,t,N
8,Base Log Reg,e,g,N
9,Base Log Reg,e,f,N


In [50]:
log_reg_aug_model = joblib.load('../../model/my_models/log_reg_augmented_model.pkl')

In [51]:
model_pred('Base Log Reg Aug', log_reg_aug_model, base_lr_df)

,Model,Actual Letter,Predicted Letter,Correct
0,Base Log Reg,a,t,N
1,Base Log Reg,a,v,N
2,Base Log Reg,b,s,N
3,Base Log Reg,b,i,N
4,Base Log Reg,c,h,N
...,...,...,...,...
91,Base Log Reg Aug,w,g,N
92,Base Log Reg Aug,x,o,N
93,Base Log Reg Aug,x,g,N
94,Base Log Reg Aug,y,g,N


## Logistic Regression with Feature Engineering Testing

---

In [52]:
log_reg_with_fe = joblib.load('../../model/my_models/log_reg_with_fe.pkl')

In [53]:
model_pred('Feature Eng Log Reg', log_reg_with_fe, fe_lr_df)

,Model,Actual Letter,Predicted Letter,Correct
0,Feature Eng Log Reg,a,h,N
1,Feature Eng Log Reg,a,g,N
2,Feature Eng Log Reg,b,o,N
3,Feature Eng Log Reg,b,r,N
4,Feature Eng Log Reg,c,d,N
5,Feature Eng Log Reg,c,h,N
6,Feature Eng Log Reg,d,d,Y
7,Feature Eng Log Reg,d,y,N
8,Feature Eng Log Reg,e,y,N
9,Feature Eng Log Reg,e,g,N


In [54]:
log_reg_aug_model = joblib.load('../../model/my_models/log_reg_fe_augmented_model.pkl')

In [55]:
model_pred('Feature Eng Log Reg Aug', log_reg_aug_model, fe_lr_df)

,Model,Actual Letter,Predicted Letter,Correct
0,Feature Eng Log Reg,a,h,N
1,Feature Eng Log Reg,a,g,N
2,Feature Eng Log Reg,b,o,N
3,Feature Eng Log Reg,b,r,N
4,Feature Eng Log Reg,c,d,N
...,...,...,...,...
91,Feature Eng Log Reg Aug,w,f,N
92,Feature Eng Log Reg Aug,x,c,N
93,Feature Eng Log Reg Aug,x,c,N
94,Feature Eng Log Reg Aug,y,a,N


## CNN Testing
----

In [56]:
cnn_model = load_model('../../model/my_models/CNN_model.h5')

In [57]:
model_pred('CNN', cnn_model,cnn_df)

1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 8ms/step


,Model,Actual Letter,Predicted Letter,Correct
0,CNN,a,p,N
1,CNN,a,q,N
2,CNN,b,x,N
3,CNN,b,l,N
4,CNN,c,l,N
5,CNN,c,x,N
6,CNN,d,l,N
7,CNN,d,l,N
8,CNN,e,y,N
9,CNN,e,k,N


In [58]:
cnn_aug_model = load_model('../../model/my_models/CNN_augmented_model.h5')

In [59]:
model_pred('CNN_aug', cnn_aug_model,cnn_df)

1/1 [==============================] - 0s 9ms/step


,Model,Actual Letter,Predicted Letter,Correct
0,CNN,a,p,N
1,CNN,a,q,N
2,CNN,b,x,N
3,CNN,b,l,N
4,CNN,c,l,N
...,...,...,...,...
91,CNN_aug,w,l,N
92,CNN_aug,x,c,N
93,CNN_aug,x,o,N
94,CNN_aug,y,c,N


## Result Evaluation 
---

### Logistic Regression

In [60]:
base_lr_df[base_lr_df['Correct'] == 'Y']

,Model,Actual Letter,Predicted Letter,Correct
45,Base Log Reg,x,x,Y
47,Base Log Reg,y,y,Y
61,Base Log Reg Aug,g,g,Y
69,Base Log Reg Aug,l,l,Y
79,Base Log Reg Aug,q,q,Y


**Comment**

Base logistic regression model perfroms very poorly on the real world dataset, this is likely due to the fact that real world data is harder to classify using linear models. In the dataset I purposely performed augmentation to replicate scenarios that will happen in the real world - the basic logisitic regression model does not handle this well.

### Logisitic Regression with Feature Engineering 

In [61]:
fe_lr_df[fe_lr_df['Correct'] == 'Y']

,Model,Actual Letter,Predicted Letter,Correct
6,Feature Eng Log Reg,d,d,Y
10,Feature Eng Log Reg,f,f,Y
13,Feature Eng Log Reg,g,g,Y
14,Feature Eng Log Reg,h,h,Y
15,Feature Eng Log Reg,h,h,Y
18,Feature Eng Log Reg,k,k,Y
26,Feature Eng Log Reg,o,o,Y
27,Feature Eng Log Reg,o,o,Y
46,Feature Eng Log Reg,y,y,Y
47,Feature Eng Log Reg,y,y,Y


Instead of looking at raw pixel values, feature extraction allows us to manually pick out features from the images such as edges and patterns. This helps models generalise better as looking for features opposed to pixel values is more robust especially when noise is introduced into the dataset. This is reflected in the results we see here, feature extraction on the real world dataset has improved the performance of the logistic regression model.


### CNN

In [62]:
cnn_df[cnn_df['Correct']=='Y']

,Model,Actual Letter,Predicted Letter,Correct
15,CNN,h,h,Y
25,CNN,n,n,Y
30,CNN,q,q,Y
45,CNN,x,x,Y
52,CNN_aug,c,c,Y
59,CNN_aug,f,f,Y
68,CNN_aug,l,l,Y
74,CNN_aug,o,o,Y
75,CNN_aug,o,o,Y
78,CNN_aug,q,q,Y


I expected the CNN to show the best performance on the real world data out of all three models. Perhaps the results we saw earlier show some overfitting and the need to apply more regualrisation on the network to imporve its ability to generalise.

More investigation needs to be done here to understand why CNN is not performing thes best.

NOTE: I will come back to this notebook as how I have performed the testing is rather messy and believe there is a better way to do this.